#hide
## Doing

## To Do
 - [ ] add percentage change columns to normalize data, only challenge is counties with small populations could have a large percentage but not be as news worthy
 - [ ] pull data from JHU
 - [ ] pull data directly from usafacts 
 - [ ] Allow run in COLAB and save to Google Drive

## Done
 - [x] x day rolling averages
 - [x] ensure proper data types in df_combined_tidy
 - [x] make tidy data one row per observation (observations are by date)
 - [x] rename variable column 'Date' in df_combined_tidy


In [1]:
#hide
import pandas as pd
import urllib3 as urllib
import json
import math
from datetime import datetime
from IPython.core.display import HTML


import IPython.display
import altair as alt

pd.options.display.max_columns = None # Used when you have a lot of columns and want them all displayed

In [2]:
#hide
jhu_confirmed_df= pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

# PICK ONE AND COMMENT OUT THE OTHER (local or GIT)

# READ from Local Files
# usafacts_confirmed_df= pd.read_csv('County-Data/covid_confirmed_usafacts.csv')
# usafacts_population_df= pd.read_csv('County-Data/covid_county_population_usafacts.csv')

0# READ from GIT Files - Really this should be read from source that is updated regularly
usafacts_confirmed_df= pd.read_csv('https://github.com/JaviThomas/Covid-County-LTC-Analysis/blob/master/County-Analysis/County-Data/covid_confirmed_usafacts.csv?raw=true')
usafacts_population_df= pd.read_csv('https://github.com/JaviThomas/Covid-County-LTC-Analysis/blob/master/County-Analysis/County-Data/covid_county_population_usafacts.csv?raw=true')

#hide
# 2 - Merge Confirmed Cases data with Population Data
Good Article on how to combine data sources [Combining Dataset](https://jakevdp.github.io/PythonDataScienceHandbook/03.06-concat-and-append.html)

In [3]:
#hide
df_confirmed = usafacts_confirmed_df[usafacts_confirmed_df['countyFIPS'] > 2].copy()
df_population = usafacts_population_df[usafacts_population_df['countyFIPS'] > 2].copy()

# Set Indices to facilitate pandas concat
df_population.set_index('countyFIPS', inplace=True)
df_confirmed.set_index('countyFIPS', inplace=True)

df_confirmed_combined= pd.concat ([df_confirmed, df_population['population']], axis=1).copy()
df_confirmed_combined.reset_index(inplace=True)
# REMOVE "Counties" where population is ZERO
df_cases_clean = df_confirmed_combined[df_confirmed_combined.population != 0]
df_cases_clean.head(5)

,countyFIPS,County Name,State,stateFIPS,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20,3/18/20,3/19/20,3/20/20,3/21/20,3/22/20,3/23/20,3/24/20,3/25/20,3/26/20,3/27/20,3/28/20,3/29/20,3/30/20,3/31/20,4/1/20,4/2/20,4/3/20,4/4/20,4/5/20,4/6/20,4/7/20,4/8/20,4/9/20,4/10/20,4/11/20,4/12/20,4/13/20,4/14/20,4/15/20,4/16/20,4/17/20,4/18/20,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20,4/29/20,4/30/20,5/1/20,5/2/20,5/3/20,5/4/20,5/5/20,5/6/20,5/7/20,5/8/20,5/9/20,5/10/20,5/11/20,5/12/20,5/13/20,5/14/20,5/15/20,5/16/20,5/17/20,5/18/20,5/19/20,5/20/20,5/21/20,5/22/20,5/23/20,5/24/20,5/25/20,5/26/20,5/27/20,5/28/20,5/29/20,5/30/20,5/31/20,6/1/20,6/2/20,6/3/20,6/4/20,6/5/20,6/6/20,6/7/20,6/8/20,6/9/20,6/10/20,6/11/20,6/12/20,6/13/20,6/14/20,6/15/20,6/16/20,6/17/20,6/18/20,6/19/20,6/20/20,6/21/20,6/22/20,6/23/20,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20,7/4/20,7/5/20,7/6/20,7/7/20,7/8/20,7/9/20,7/10/20,7/11/20,7/12/20,7/13/20,7/14/20,7/15/20,7/16/20,7/17/20,7/18/20,7/19/20,7/20/20,7/21/20,7/22/20,7/23/20,7/24/20,7/25/20,7/26/20,7/27/20,7/28/20,7/29/20,7/30/20,7/31/20,8/1/20,8/2/20,8/3/20,8/4/20,8/5/20,8/6/20,8/7/20,8/8/20,8/9/20,8/10/20,8/11/20,8/12/20,8/13/20,8/14/20,8/15/20,8/16/20,8/17/20,8/18/20,8/19/20,8/20/20,8/21/20,8/22/20,8/23/20,8/24/20,8/25/20,8/26/20,8/27/20,8/28/20,8/29/20,8/30/20,8/31/20,9/1/20,9/2/20,9/3/20,9/4/20,9/5/20,9/6/20,9/7/20,9/8/20,9/9/20,9/10/20,9/11/20,9/12/20,9/13/20,9/14/20,9/15/20,9/16/20,9/17/20,9/18/20,9/19/20,9/20/20,9/21/20,9/22/20,9/23/20,9/24/20,9/25/20,9/26/20,9/27/20,9/28/20,9/29/20,9/30/20,10/1/20,10/2/20,10/3/20,10/4/20,10/5/20,10/6/20,10/7/20,10/8/20,10/9/20,10/10/20,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20,10/21/20,10/22/20,10/23/20,10/24/20,10/25/20,10/26/20,10/27/20,10/28/20,10/29/20,10/30/20,10/31/20,11/1/20,11/2/20,11/3/20,11/4/20,11/5/20,11/6/20,11/7/20,11/8/20,11/9/20,11/10/20,11/11/20,11/12/20,11/13/20,11/14/20,11/15/20,11/16/20,11/17/20,11/18/20,11/19/20,11/20/20,11/21/20,11/22/20,11/23/20,11/24/20,11/25/20,11/26/20,11/27/20,11/28/20,11/29/20,11/30/20,12/1/20,12/2/20,12/3/20,12/4/20,12/5/20,12/6/20,12/7/20,12/8/20,12/9/20,12/10/20,12/11/20,12/12/20,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20,12/23/20,12/24/20,12/25/20,12/26/20,12/27/20,12/28/20,12/29/20,12/30/20,12/31/20,1/1/21,1/2/21,1/3/21,1/4/21,1/5/21,1/6/21,1/7/21,1/8/21,1/9/21,1/10/21,1/11/21,1/12/21,1/13/21,1/14/21,1/15/21,1/16/21,1/17/21,1/18/21,1/19/21,1/20/21,1/21/21,1/22/21,1/23/21,1/24/21,population
0,1001,Autauga County,AL,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,4,6,6,6,6,7,7,10,10,12,12,12,12,12,12,17,17,19,19,19,23,25,25,25,25,27,28,30,32,33,36,37,37,39,40,42,42,42,45,48,53,53,58,61,67,68,74,84,91,93,103,103,110,110,120,127,136,147,149,155,159,173,189,192,205,212,216,220,233,238,239,241,248,259,265,272,282,295,312,323,331,357,368,373,375,400,411,431,434,442,453,469,479,488,498,503,527,537,553,561,568,591,615,618,644,651,661,670,684,706,728,746,756,780,789,827,842,857,865,886,905,921,932,942,965,974,974,1002,1015,1030,1052,1066,1073,1073,1096,1113,1134,1215,1215,1215,1241,1250,1252,1262,1273,1274,1291,1293,1293,1293,1322,1324,1351,1355,1366,1377,1389,1400,1438,1442,1452,1452,1466,1475,1492,1498,1504,1508,1522,1544,1551,1565,1576,1585,1601,1619,1624,1664,1673,1690,1691,1714,1715,1715,1757,1764,1773,1785,1787,1791,1798,1805,1818,1828,1831,1839,1852,1863,1882,1898,1905,1911,1924,1928,1949,1966,1983,1989,1999

# 2 - Make the data "tidy" to facilitate analysis and plotting
To process data it is best if it is in [tidy data](https://vita.had.co.nz/papers/tidy-data.html)  which means each row is an observation and the columns are the dimensions and metrics of that observation.

In [4]:
#hide
# date_cols = list(df_combined_clean.columns[3:-2])
id_cols = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population'] # these are the dimensions of the data

df_cases_tidy = df_cases_clean.melt(id_vars =id_cols, value_vars = df_cases_clean.columns[4:-2], var_name='Date', value_name='Cases')

# Convert the Date column which is in a string format to a python datetime datatype.  This facilitates sorting and date differencing
df_cases_tidy['Date'] = pd.to_datetime(df_cases_tidy['Date'], format='%m/%d/%y') 

display(df_cases_tidy.dtypes) # Display data types of each column in the Dataframe
df_cases_tidy

#hide
%%time
# Sort data before save
df_cases_tidy.sort_values(by=['countyFIPS', 'Date'], inplace=True)
#Save to CSV file
#df_cases_tidy.to_csv('County-Data/usafacts_cases_tidy.csv') # TODO - Decide at what checkpoint to save

# Step 2 - Get Rolling Averages

In [6]:
#hide
%%time
first_time = True
# set temporarily to suppress "SettingWithCopyWarning: A value is trying to be set on a copy of a slice from a DataFrame"
pd.set_option('mode.chained_assignment', None) 

# Processing the data for all states takes ~4-5 minutes. That means slow feedback when debugging and trying to add new features to the data
# So we do it for one State to get fast feedback.  Once satisfied we change to process data for all states.

# UNCOMMENT THE Following 3 lines and UnComment out line following with for loop to get a dataframe with all states
# state = 'FL'
# mydf = df_cases_tidy[(df_cases_tidy['State'] == state)]
# for countyFIPS in mydf['countyFIPS'].unique():

# COMMENT below line (and comment above lines) to get data for all states
for countyFIPS in df_cases_tidy['countyFIPS'].unique():  # COMMENT this line if using single state filter for faster feedback

    tmp_df = df_cases_tidy[df_cases_tidy['countyFIPS'] == countyFIPS]
    tmp_df['NewCases'] = tmp_df['Cases'] - tmp_df['Cases'].shift(1) # shift(1) gets the column value from 1 row back (aka previous row)
    tmp_df['7day_avg'] = tmp_df['NewCases'].rolling(7).mean().round(1)
    tmp_df['14day_avg'] = tmp_df['NewCases'].rolling(14).mean().round(1)
    tmp_df['28day_avg'] = tmp_df['NewCases'].rolling(28).mean().round(1)


    if first_time:
        df_cases_7day_tidy = tmp_df.copy()
        first_time = False
        print('First')
    else:
        df_cases_7day_tidy = df_cases_7day_tidy.append(tmp_df)

pd.set_option('mode.chained_assignment', 'warn') # Turn off warning suppresion

First
Wall time: 5min 2s


In [7]:
#hide
df_cases_7day_tidy.tail(10)  # Did this to ensure my calculations worked. Change to head and see what happens

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg
1127977,Weston County,WY,56045,56,6927,2021-01-14,602,1.0,17.0,9.0,5.9
1131119,Weston County,WY,56045,56,6927,2021-01-15,602,0.0,16.7,9.0,5.7
1134261,Weston County,WY,56045,56,6927,2021-01-16,602,0.0,16.7,8.9,5.7
1137403,Weston County,WY,56045,56,6927,2021-01-17,602,0.0,16.7,8.9,5.6
1140545,Weston County,WY,56045,56,6927,2021-01-18,602,0.0,16.7,8.8,5.5
1143687,Weston County,WY,56045,56,6927,2021-01-19,606,4.0,0.7,9.1,5.4
1146829,Weston County,WY,56045,56,6927,2021-01-20,606,0.0,0.7,9.1,5.2
1149971,Weston County,WY,56045,56,6927,2021-01-21,609,3.0,1.0,9.0,5.3
1153113,Weston County,WY,56045,56,6927,2021-01-22,609,0.0,1.0,8.9,5.3
1156255,Weston County,WY,56045,56,6927,2021-01-23,609,0.0,1.0,8.9,5.3


In [8]:
#hide
%%time
# Save data - optional decide at what point you want to save a copy for sharing with others
#df_cases_7day_tidy.to_csv('df_cases_7day_tidy.tmp.csv')

Wall time: 0 ns


# 3 - Add %Diff vs yesterday, 7 days ago, 14 days ago, 28 days ago

In [9]:
#hide
# Parameterize differecing columns by date

# Dictionary of columns 
diffCols = [{'col2Diff': 'NewCases', 'daysAgo2Diff': 7},
            {'col2Diff': '7day_avg', 'daysAgo2Diff': 7},
            {'col2Diff': '14day_avg', 'daysAgo2Diff': 7},
            {'col2Diff': '28day_avg', 'daysAgo2Diff': 7}]

for item in diffCols:
    newColName = item['col2Diff'] + '_' + str(item['daysAgo2Diff']) + 'DayDiff'
    df_cases_7day_tidy[newColName] = (df_cases_7day_tidy[item['col2Diff']] - df_cases_7day_tidy[item['col2Diff']].shift(item['daysAgo2Diff'])) 

df_cases_7day_tidy.sort_values(by=['countyFIPS', 'Date']).tail(10)

,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,NewCases_7DayDiff,7day_avg_7DayDiff,14day_avg_7DayDiff,28day_avg_7DayDiff
1127977,Weston County,WY,56045,56,6927,2021-01-14,602,1.0,17.0,9.0,5.9,-3.0,16.0,7.4,3.9
1131119,Weston County,WY,56045,56,6927,2021-01-15,602,0.0,16.7,9.0,5.7,-2.0,15.4,7.2,3.7
1134261,Weston County,WY,56045,56,6927,2021-01-16,602,0.0,16.7,8.9,5.7,0.0,15.6,7.2,3.7
1137403,Weston County,WY,56045,56,6927,2021-01-17,602,0.0,16.7,8.9,5.6,0.0,15.6,7.5,3.6
1140545,Weston County,WY,56045,56,6927,2021-01-18,602,0.0,16.7,8.8,5.5,0.0,15.8,7.5,3.5
1143687,Weston County,WY,56045,56,6927,2021-01-19,606,4.0,0.7,9.1,5.4,-112.0,-16.7,-0.2,-0.5
1146829,Weston County,WY,56045,56,6927,2021-01-20,606,0.0,0.7,9.1,5.2,0.0,-16.7,0.1,-0.7
1149971,Weston County,WY,56045,56,6927,2021-01-21,609,3.0,1.0,9.0,5.3,2.0,-16.0,0.0,-0.6
1153113,Weston County,WY,56045,56,6927,2021-01-22,609,0.0,1.0,8.9,5.3,0.0,-15.7,-0.1,-0.4
1156255,Weston County,WY,56045,56,6927,2021-01-23,609,0.0,1.0,8.9,5.3,0.0,-15.7,0.0,-0.4


# 4 - Get data from latest date to get top 10's

In [10]:
#hide_input
# Get DataFrame with only latest date
df_cases_7day_tidy_latest = df_cases_7day_tidy[df_cases_7day_tidy['Date'] == df_cases_7day_tidy['Date'].max()].copy()

# top10Fields = ['NewCases_7DayDiff', 'NewCases_14DayDiff', 'NewCases_28DayDiff']
top10Fields = [{'colName': '7day_avg_7DayDiff', 'colTitle': '<7 Day Avg.> - <7 Day Avg. from 7 days ago>'},
               {'colName': '14day_avg_7DayDiff', 'colTitle': '<14 Day Avg.>. - <14 Day Avg. from 7 days ago>'},
               {'colName': '28day_avg_7DayDiff', 'colTitle': '<28 Day Avg.> - <28 Day Avg. from 7 days ago>'}]

cols2show = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population', 'Date', 'Cases', 'NewCases', '7day_avg', '14day_avg', '28day_avg']
for field in top10Fields:
    cols2show.append(field['colName'])
    display(HTML('<h2>Top 10 Counties with <b>&#8607;</b> Rising Rates  <b>&#8607;</b> by: ' + field['colTitle'] + '</h2>'))
    display(df_cases_7day_tidy_latest.sort_values(by=field['colName'], ascending=False)[cols2show].head(10)) # Top 10 Rising
    
    display(HTML('<h2>Top 10 Counties with <b>&#8609;</b> Falling Rates <b>&#8609;</b> by: ' + field['colTitle'] + '</h2>'))
    display(df_cases_7day_tidy_latest.sort_values(by=field['colName'])[cols2show].head(10)) # Top 10 Falling
    
    cols2show = ['County Name', 'State', 'countyFIPS', 'stateFIPS', 'population', 'Date', 'Cases', 'NewCases', '7day_avg', '14day_avg', '28day_avg']


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,7day_avg_7DayDiff
1155876,Webb County,TX,48479,48,276652,2021-01-23,37761,607.0,729.0,645.7,437.4,166.6
1156006,Prince William County,VA,51153,51,470335,2021-01-23,33048,174.0,432.1,355.9,312.7,152.4
1155986,Loudoun County,VA,51107,51,413538,2021-01-23,18151,342.0,272.3,204.6,168.2,135.3
1155068,New Hanover County,NC,37129,37,234473,2021-01-23,12691,127.0,199.0,148.5,135.0,101.0
1153567,Gwinnett County,GA,13135,13,936250,2021-01-23,69689,1086.0,878.6,828.6,752.7,100.0
1155697,Denton County,TX,48121,48,887207,2021-01-23,49600,655.0,540.6,492.3,474.9,96.6
1155471,Spartanburg County,SC,45083,45,319785,2021-01-23,27778,303.0,413.0,369.0,332.2,88.0
1156033,Chesapeake City,VA,51550,51,244835,2021-01-23,14248,233.0,325.9,287.4,191.4,76.9
1155954,Chesterfield County,VA,51041,51,352802,2021-01-23,17542,134.0,253.3,215.1,186.5,76.4
1155384,Forest County,PA,42053,42,7247,2021-01-23,1246,0.0,94.4,59.6,34.5,69.7


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,7day_avg_7DayDiff
1153318,Los Angeles County,CA,6037,6,10039107,2021-01-23,1031177,8122.0,8885.4,11427.8,13056.3,-5084.7
1153335,San Bernardino County,CA,6071,6,2180085,2021-01-23,259121,1367.0,1848.4,2489.6,2761.0,-1282.3
1153217,Maricopa County,AZ,4013,4,4485414,2021-01-23,444431,4688.0,4449.9,4977.9,5876.6,-1056.1
1153329,Orange County,CA,6059,6,3175692,2021-01-23,220384,1517.0,1893.6,2412.6,2708.6,-1038.0
1155737,Harris County,TX,48201,48,4713325,2021-01-23,294266,995.0,2279.1,2730.0,2460.8,-901.8
1153336,San Diego County,CA,6073,6,3338330,2021-01-23,222576,2847.0,2244.0,2679.7,2950.6,-871.4
1153332,Riverside County,CA,6065,6,2470546,2021-01-23,252440,1433.0,2457.9,2879.9,3062.5,-844.0
1155693,Dallas County,TX,48113,48,2635516,2021-01-23,243726,2065.0,1898.1,2281.9,2296.7,-767.6
1154862,Clark County,NV,32003,32,2266715,2021-01-23,207240,1314.0,1140.0,1381.9,1562.7,-483.9
1153342,Santa Clara County,CA,6085,6,1927852,2021-01-23,96435,407.0,904.6,1133.9,1187.9,-458.5


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,14day_avg_7DayDiff
1155876,Webb County,TX,48479,48,276652,2021-01-23,37761,607.0,729.0,645.7,437.4,240.8
1155651,Bexar County,TX,48029,48,2003554,2021-01-23,159519,1684.0,1882.3,2076.6,1883.1,148.9
1154982,Queens County,NY,36081,36,2253858,2021-01-23,162455,1792.0,1805.6,1777.6,1625.7,120.2
1156033,Chesapeake City,VA,51550,51,244835,2021-01-23,14248,233.0,325.9,287.4,191.4,113.0
1154965,Kings County,NY,36047,36,2559903,2021-01-23,158856,1596.0,1657.7,1706.6,1547.5,103.5
1154944,Bronx County,NY,36005,36,1418207,2021-01-23,106586,1227.0,1079.4,1084.4,974.2,87.2
1156006,Prince William County,VA,51153,51,470335,2021-01-23,33048,174.0,432.1,355.9,312.7,81.3
1153567,Gwinnett County,GA,13135,13,936250,2021-01-23,69689,1086.0,878.6,828.6,752.7,79.1
1155986,Loudoun County,VA,51107,51,413538,2021-01-23,18151,342.0,272.3,204.6,168.2,78.6
1155704,Ector County,TX,48135,48,166223,2021-01-23,13236,209.0,144.9,120.9,63.3,72.4


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,14day_avg_7DayDiff
1153318,Los Angeles County,CA,6037,6,10039107,2021-01-23,1031177,8122.0,8885.4,11427.8,13056.3,-2386.6
1153217,Maricopa County,AZ,4013,4,4485414,2021-01-23,444431,4688.0,4449.9,4977.9,5876.6,-882.7
1153336,San Diego County,CA,6073,6,3338330,2021-01-23,222576,2847.0,2244.0,2679.7,2950.6,-662.9
1153329,Orange County,CA,6059,6,3175692,2021-01-23,220384,1517.0,1893.6,2412.6,2708.6,-565.2
1153335,San Bernardino County,CA,6071,6,2180085,2021-01-23,259121,1367.0,1848.4,2489.6,2761.0,-548.3
1154862,Clark County,NV,32003,32,2266715,2021-01-23,207240,1314.0,1140.0,1381.9,1562.7,-448.5
1153332,Riverside County,CA,6065,6,2470546,2021-01-23,252440,1433.0,2457.9,2879.9,3062.5,-445.9
1154339,Middlesex County,MA,25017,25,1611699,2021-01-23,97177,846.0,747.3,861.8,976.2,-435.5
1153724,Cook County,IL,17031,17,5150233,2021-01-23,443600,1943.0,1745.0,1972.1,2147.1,-416.7
1154335,Essex County,MA,25009,25,789034,2021-01-23,74259,513.0,525.4,623.1,742.1,-408.9


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,28day_avg_7DayDiff
1153217,Maricopa County,AZ,4013,4,4485414,2021-01-23,444431,4688.0,4449.9,4977.9,5876.6,1011.9
1155651,Bexar County,TX,48029,48,2003554,2021-01-23,159519,1684.0,1882.3,2076.6,1883.1,257.3
1153318,Los Angeles County,CA,6037,6,10039107,2021-01-23,1031177,8122.0,8885.4,11427.8,13056.3,248.9
1153220,Pima County,AZ,4019,4,1047279,2021-01-23,95720,1023.0,977.4,1077.0,1307.3,206.2
1154982,Queens County,NY,36081,36,2253858,2021-01-23,162455,1792.0,1805.6,1777.6,1625.7,170.0
1155908,Salt Lake County,UT,49035,49,1160437,2021-01-23,126093,625.0,621.7,705.2,959.2,131.4
1154965,Kings County,NY,36047,36,2559903,2021-01-23,158856,1596.0,1657.7,1706.6,1547.5,123.9
1155737,Harris County,TX,48201,48,4713325,2021-01-23,294266,995.0,2279.1,2730.0,2460.8,117.0
1154944,Bronx County,NY,36005,36,1418207,2021-01-23,106586,1227.0,1079.4,1084.4,974.2,113.0
1155095,Wake County,NC,37183,37,1111761,2021-01-23,60547,675.0,615.1,708.6,766.9,112.0


,County Name,State,countyFIPS,stateFIPS,population,Date,Cases,NewCases,7day_avg,14day_avg,28day_avg,28day_avg_7DayDiff
1156187,Jackson County,WI,55053,55,20643,2021-01-23,2562,3.0,5.0,4.4,-742.8,-752.8
1155693,Dallas County,TX,48113,48,2635516,2021-01-23,243726,2065.0,1898.1,2281.9,2296.7,-592.8
1153329,Orange County,CA,6059,6,3175692,2021-01-23,220384,1517.0,1893.6,2412.6,2708.6,-409.5
1153335,San Bernardino County,CA,6071,6,2180085,2021-01-23,259121,1367.0,1848.4,2489.6,2761.0,-400.7
1155856,Tarrant County,TX,48439,48,2102515,2021-01-23,203174,1864.0,2187.3,2317.4,2406.5,-400.3
1153332,Riverside County,CA,6065,6,2470546,2021-01-23,252440,1433.0,2457.9,2879.9,3062.5,-300.8
1155814,Nueces County,TX,48355,48,362294,2021-01-23,32635,146.0,160.1,182.5,176.5,-248.2
1155857,Taylor County,TX,48441,48,138034,2021-01-23,13630,88.0,91.4,125.6,106.5,-199.5
1155697,Denton County,TX,48121,48,887207,2021-01-23,49600,655.0,540.6,492.3,474.9,-192.7
1155862,Tom Green County,TX,48451,48,119200,2021-01-23,11847,58.0,60.0,70.9,87.8,-187.0


# Display Charts for Counties to compare different measures to detect trends

In [11]:
#hide_input
date_time_str = '2020-10-01'
date_time_obj = datetime.strptime(date_time_str, '%Y-%m-%d')

top10Fields = [{'colName': '7day_avg_7DayDiff', 'colTitle': '<7 Day Avg.> - <7 Day Avg. from 7 days ago>'},
               {'colName': '14day_avg_7DayDiff', 'colTitle': '<14 Day Avg.>. - <14 Day Avg. from 7 days ago>'},
               {'colName': '28day_avg_7DayDiff', 'colTitle': '<28 Day Avg.> - <28 Day Avg. from 7 days ago>'}]

# Goal is to print the 3 charts (one per field) in a row per FIPS
topN = 3
for fieldSort in top10Fields:
#     display(HTML('<H1>Top ' + str(topN) + ' Counties by: ' + field['colTitle'] + '</H1><BR>'))
    display(HTML('<h2>Top ' + str(topN) + ' Counties with <b>&#8607;</b> Rising Rates  <b>&#8607;</b> by: ' + fieldSort['colTitle'] + '</h2>'))
    for countyFIPS in df_cases_7day_tidy_latest.sort_values(by=fieldSort['colName'], ascending=False)['countyFIPS'].head(topN):
        charts = []
        for field in top10Fields:
            state_df = df_cases_7day_tidy[(df_cases_7day_tidy['countyFIPS'] == countyFIPS) & (df_cases_7day_tidy['Date'] >= date_time_obj )]
            row = state_df.iloc[0]
            #display(row)
            chart_title = row['State'] + ': ' + row['County Name'] + ' - Population: ' + f"{row['population']:,}"
            
            chart1 = alt.Chart(state_df).mark_bar().encode(
                                 #x=alt.X('yearmonthdate(Date):T', title='date', axis=alt.Axis(labelSeparation=2)),
                                 x=alt.X('Date:T', title='date', axis=alt.Axis(labelSeparation=2)),
                                 y=alt.Y(field['colName']+':Q', title=field['colTitle']),
                                 ).properties( width=400, height=400, title=chart_title)
            charts.append(chart1)
        display(charts[0] | charts[1] | charts[2])
    display(HTML('<HR>'))
    
    display(HTML('<h2>Top ' + str(topN) + ' Counties with <b>&#8609;</b> Falling Rates <b>&#8609;</b> by: ' + fieldSort['colTitle'] + '</h2>'))
    for countyFIPS in df_cases_7day_tidy_latest.sort_values(by=fieldSort['colName'])['countyFIPS'].head(topN):
        charts = []
        for field in top10Fields:
            state_df = df_cases_7day_tidy[(df_cases_7day_tidy['countyFIPS'] == countyFIPS) & (df_cases_7day_tidy['Date'] >= date_time_obj )]
            row = state_df.iloc[0]
            #display(row)
            chart_title = row['State'] + ': ' + row['County Name'] + ' - Population: ' + f"{row['population']:,}"
            
            chart1 = alt.Chart(state_df).mark_bar().encode(
                                 #x=alt.X('yearmonthdate(Date):T', title='date', axis=alt.Axis(labelSeparation=2)),
                                 x=alt.X('Date:T', title='date', axis=alt.Axis(labelSeparation=2)),
                                 y=alt.Y(field['colName']+':Q', title=field['colTitle']),
                                 ).properties( width=400, height=400, title=chart_title)
            charts.append(chart1)
        display(charts[0] | charts[1] | charts[2])
    display(HTML('<HR>'))

    
# display(HTML('<h2>Top 10 Counties with <b>&#8607;</b> Rising Rates  <b>&#8607;</b> by: ' + field['colTitle'] + '</h2>'))
# display(df_cases_7day_tidy_latest.sort_values(by=field['colName'], ascending=False)[cols2show].head(10)) # Top 10 Rising

# display(df_cases_7day_tidy_latest.sort_values(by=field['colName'])[cols2show].head(10)) # Top 10 Falling   

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

...

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

alt.HConcatChart(...)

# Play Area - WARNING this code MAY not work as I would expect

In [12]:
row = state_df.iloc[0]
#display(row)
row['State'] + ': ' + row['County Name'] + ', Population: ' + f"{row['population']:,}"
#f"{row['population']:,}"

'CA: Orange County, Population: 3,175,692'

In [13]:
# OKAY I didn't make the data "Fully tidy"  if I did I could use the facet option instead of for loops
# ====================================================================================================

from vega_datasets import data
iris = data.iris()

alt.Chart(iris).mark_point().encode(
    x='petalLength:Q',
    y='petalWidth:Q',
    color='species:N'
).properties(
    width=180,
    height=180
).facet(
    facet='species:N',
    columns=2
)

alt.FacetChart(...)

In [14]:
iris

,sepalLength,sepalWidth,petalLength,petalWidth,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


# PLAY AREA